# BITS F312 - Neural Network and Fuzzy Logic



# NNFL Assignment 2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Changing directory to the directory containing dataset
%cd drive/MyDrive/NNFL/Data_A2/

/content/drive/MyDrive/NNFL/Data_A2


In [ ]:
# listing datasets
%ls -l

total 1234719
-rw------- 1 root root     637638 Oct 31 04:55 Assignment2.pdf
-rw------- 1 root root        259 Oct 31 04:57 class_label.mat
-rw------- 1 root root      40295 Oct 31 04:57 data55.xlsx
-rw------- 1 root root      21269 Oct 31 04:55 data5.xlsx
-rw------- 1 root root 1263647365 Oct 31 04:58 input.mat
drwx------ 2 root root       4096 Nov 18 05:58 logs/


In [ ]:
# libraries required
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [ ]:
# supressing warnings
import warnings
warnings.filterwarnings('ignore')

# Q2 
Implement kernel perceptron algorithm for the classification task. The dataset (data55.xlsx) contains 19
features and the last column is the output (class label). You can use hold-out cross-validation (70, 10, and
20%) for the selection of training, validation and test instances. Evaluate accuracy, sensitivity and
specificity measures for the evaluation of test instances. Evaluate the classification performance
separately using linear, RBF, and polynomial kernels (Packages such as Scikitlearn, keras, tensorflow,
pytorch etc. are not allowed).


In [ ]:
def metrics(Y_true, Y_pred):
    FP=0 # For counting the False Positives
    FN=0 # For counting the False Negatives
    TN=0 # For counting the True Negatives
    TP=0 # For counting the True Positives

    for i in range(len(Y_true)):
      if Y_true[i]==1:
        if Y_pred[i]==1:
          TP+=1
        else:
          FN+=1
      else:
        if Y_pred[i]==0:
          TN+=1
        else:
          FP+=1

    print('{}'.format('-'*75))
    
    sens= TP/(TP+FN)
    spes = TN/(TN+FP)

    print("Sensitivity : ", sens)
    print("Specificity : ", spes)
    print("Accuracy ((TN+TP)/(TN+TP+FN+FP)) : ", ((TP+TN)/(TN+FN+TP+FP)))

In [ ]:
def polynomialKernel(idx_list, idx, lower, upper, deg):
    idx = [*idx]
    if deg == 0:
        idx_list.append(idx)
        return
    for i in range(lower, upper):
        idx.append(i)
        polynomialKernel(idx_list, idx, i, upper, deg-1)
        idx = idx[0:-1]

def polynomialFeats(X, deg):
    row, col = X.shape
    feats = []
    for i in range(1, deg+1):
        l = []
        polynomialKernel(l, [], 0, col, i)
        for idx in l:
            x_temp = np.ones((row,))
            for idx in idx:
                x_temp = x_temp * X[:, idx]
            feats.append(x_temp)
    return np.stack(feats, axis=1)


In [ ]:
def resultQ2(filename = 'data55.xlsx'):
  dataset = pd.read_excel(filename, header = None)

  row, col = dataset.shape
  feats = col - 1 

  # normalization
  dataset.loc[:, dataset.columns != feats] = (dataset.loc[:, dataset.columns != feats]-dataset.loc[:, dataset.columns != feats].mean(axis=0))/dataset.loc[:, dataset.columns != feats].std(axis=0)
  
  # spliting dataset into train test and val
  training_data, validation_data, testing_data = np.split(dataset.sample(frac=1),[int(0.7*len(dataset)), int(0.8*len(dataset))])

  training_data = np.array(training_data)
  validation_data = np.array(validation_data)
  testing_data = np.array(testing_data)
  training_data_X = polynomialFeats(training_data[:, :feats], 4)
  training_data_y = training_data[:, feats]
  validation_data_X = polynomialFeats(validation_data[:, :feats], 4)
  validation_data_y = validation_data[:, feats]
  testing_data_X = polynomialFeats(testing_data[:, :feats], 4)
  testing_data_y = testing_data[:, feats]

  train_row, train_col = training_data_X.shape

  W = np.random.randn(train_col)  
  bias = np.ones(train_row)

  W, bias = perceptron(training_data_X, training_data_y, bias, W)
  print("The Weights after training is as follows: \n")
  pprint(W)
  print("The Bias after training is as follows: ", bias[0])
  print('{}'.format('-'*75))

  train_pred = pred_eval(training_data_X, W, bias)
  train_pred = np.where(train_pred > 0.5, 1,0)
  print("Training Accuracy: \n")
  metrics(train_pred, training_data_y)
  print('{}'.format('-'*75))

  test_pred = pred_eval(testing_data_X, W, bias)
  test_pred = np.where(test_pred > 0.5, 1,0)
  print("Testing Accuracy: \n")
  metrics(test_pred, testing_data_y)
  print('{}'.format('-'*75))
  
  validation_pred = pred_eval(validation_data_X, W, bias)
  validation_pred = np.where(validation_pred > 0.5, 1,0)
  print("Validation Accuracy: \n")
  metrics(validation_pred, validation_data_y)
  print('{}'.format('-'*75))


In [ ]:
resultQ2()

The Weights after training is as follows: 

array([ 0.13049155, -1.31189354, -0.67991826, ...,  0.57346655,
       -0.80864538,  0.11325281])
The Bias after training is as follows:  1.0155357007732724
---------------------------------------------------------------------------
Training Accuracy: 

Sensitivity :  0.9456521739130435
Specificity :  0.8906256289423758
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.9230769230769231
---------------------------------------------------------------------------
Testing Accuracy: 

Sensitivity :  0.8765432098765432
Specificity :  0.9523809523809523
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.9254385964912281
---------------------------------------------------------------------------
Validation Accuracy: 

Sensitivity :  0.8833333333333333
Specificity :  0.8414634146341463
Accuracy ((TN+TP)/(TN+TP+FN+FP)) :  0.8591549295774648
---------------------------------------------------------------------------
